In [14]:
#encoding: utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [15]:
# df1~ is DataFrame datatype
df1 = pd.read_csv('data/user_behaviour_201809_01.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
df2 = pd.read_csv('data/user_behaviour_201809_02.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
df3 = pd.read_csv('data/user_behaviour_201809_03.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
df4 = pd.read_csv('data/user_behaviour_201809_04.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
df5 = pd.read_csv('data/user_behaviour_201809_05.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
df6 = pd.read_csv('data/user_behaviour_201809_06.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
df7 = pd.read_csv('data/user_behaviour_201809_07.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
# df8 = pd.read_csv('data/user_behaviour_201810_01.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
# df9 = pd.read_csv('data/user_behaviour_201810_02.csv', sep=',', header=0, parse_dates=["使用时间"], encoding='gb2312')
print(type(df1))
print(df1.head(10),'\n', 'df1 first 10 elements')
frames = [df1, df2, df3, df4, df5, df6, df7]
data = pd.concat(frames)
# concat是对几个数据集的拼接
# overall_information，count，mean，std，min，max
data.describe()
# data.shape

<class 'pandas.core.frame.DataFrame'>
           IMEI        日期           APP名称                使用时间  使用时长  \
0  8633****5756  20180901           搜狗输入法 2018-09-01 00:01:00     4   
1  8660****9090  20180901  GnssPowerSaver 2018-09-01 00:01:00     5   
2  A000****6B5D  20180901            塔读文学 2018-09-01 00:00:00     9   
3  A000****6B5D  20180901          华为应用市场 2018-09-01 00:01:00     2   
4  A000****6B5D  20180901            搜狐视频 2018-09-01 00:01:00    10   
5  A100****9464  20180901            手机管家 2018-09-01 00:03:00     2   
6  8642****0853  20180901           章鱼输入法 2018-09-01 00:03:00     4   
7  A000****B5DE  20180901           电信营业厅 2018-09-01 00:03:00     2   
8  9900****7669  20180901            系统桌面 2018-09-01 00:04:00    11   
9  A100****1FA9  20180901              任务 2018-09-01 00:00:00   148   

        使用流量  GPS经度  GPS纬度   省   市             手机型号 注册省 注册市  
0        NaN    NaN    NaN  上海  上海         OPPO R9s  上海  上海  
1        NaN    NaN    NaN  上海  上海        OPPO A37m  上海 

,日期,使用时间,使用时长,使用流量,GPS经度,GPS纬度
count,4.229315e+06,4229315,4.229315e+06,9.539850e+05,4384.000000,4384.000000
mean,2.018091e+07,2018-09-14 03:11:14.725573632,5.165055e+02,4.693946e+06,121.412267,31.214000
min,2.018090e+07,2018-09-01 00:00:00,1.000000e+00,0.000000e+00,120.429175,30.658342
25%,2.018091e+07,2018-09-06 22:24:02,3.000000e+00,2.236500e+04,121.300736,31.135859
50%,2.018091e+07,2018-09-13 13:44:19,1.100000e+01,1.508650e+05,121.423319,31.219134
75%,2.018092e+07,2018-09-20 16:53:53.500000,6.400000e+01,6.781800e+05,121.507124,31.287088
max,2.018093e+07,2018-09-30 23:59:49,1.771110e+06,2.502329e+10,122.020567,31.890294
std,8.153471e+00,NaN,4.360917e+03,6.026038e+07,0.167781,0.159696


In [16]:
def preprocess(cdr, GPS=False):
    if not GPS:
        columns_to_drop = ['GPS经度', 'GPS纬度', '省', '市']
        cdr.drop(columns_to_drop, axis=1, inplace=True)
        cdr = cdr[['IMEI',"日期",'使用时间','使用流量',"APP名称","使用时长"]]
    else:
        cdr = cdr[['IMEI',"日期",'使用时间','使用流量',"APP名称","GPS经度","GPS纬度"]]
        cdr = cdr.rename(columns={"GPS经度": "longitude", "GPS纬度": "latitude"})
    cdr['使用时间'] = pd.to_datetime(cdr['使用时间'])
# to_datetime转换日期时间表达格式
    cdr = cdr.rename(columns={"日期":'date',"使用时间": "ds", "使用流量": "y","APP名称":"APP","使用时长":"time"})
    return cdr

def feature_extraction(traffic,t_index):
    traffic_resampled = traffic.groupby(['IMEI']).apply(lambda x: x.resample('1h', on='ds').sum().reindex(t_index).fillna(0))
    traffic_resampled.reset_index(inplace=True)
    traffic_resampled = traffic_resampled.rename(columns={"level_1": "ds"})
    traffic_resampled = traffic_resampled.set_index(['IMEI', 'ds'])
    traffic_resampled = traffic_resampled.unstack(level=-1)
    return traffic_resampled

In [17]:
# cdr = preprocess(octe)
cdr = preprocess(data)
cdr = cdr.fillna(0)
# 空数据赋0
cdr.y = np.log10(cdr.y+1)
# 对流量数据取对数运算

C:\Users\Chenting Zhang\AppData\Local\Temp\ipykernel_28268\1544606722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdr['使用时间'] = pd.to_datetime(cdr['使用时间'])


## Feature extraction

In [19]:
traffic = cdr[['IMEI','ds','y']]
# 取'IMEI','ds','y'三列数据
print(traffic)
# 运行时间：5min以内

                IMEI                  ds    y
0       8633****5756 2018-09-01 00:01:00  0.0
1       8660****9090 2018-09-01 00:01:00  0.0
2       A000****6B5D 2018-09-01 00:00:00  0.0
3       A000****6B5D 2018-09-01 00:01:00  0.0
4       A000****6B5D 2018-09-01 00:01:00  0.0
...              ...                 ...  ...
509001  A000****3FED 2018-09-30 20:59:44  0.0
509002  A000****3FED 2018-09-30 22:31:13  0.0
509003  A000****3FED 2018-09-30 23:24:34  0.0
509004  A000****3FED 2018-09-30 23:33:51  0.0
509005  A100****5DF2 2018-09-30 21:07:30  0.0

[4229315 rows x 3 columns]


In [21]:
t_index = pd.date_range(start='2018-09-01', end='2018-09-30 23:00:00', freq='1h')
print(t_index)
# date_range用于生成一个固定频率的时间索引
#t_index = pd.date_range(start='2018-10-01', end='2018-10-31 23:00:00', freq='1h')

DatetimeIndex(['2018-09-01 00:00:00', '2018-09-01 01:00:00',
               '2018-09-01 02:00:00', '2018-09-01 03:00:00',
               '2018-09-01 04:00:00', '2018-09-01 05:00:00',
               '2018-09-01 06:00:00', '2018-09-01 07:00:00',
               '2018-09-01 08:00:00', '2018-09-01 09:00:00',
               ...
               '2018-09-30 14:00:00', '2018-09-30 15:00:00',
               '2018-09-30 16:00:00', '2018-09-30 17:00:00',
               '2018-09-30 18:00:00', '2018-09-30 19:00:00',
               '2018-09-30 20:00:00', '2018-09-30 21:00:00',
               '2018-09-30 22:00:00', '2018-09-30 23:00:00'],
              dtype='datetime64[ns]', length=720, freq='H')


In [22]:
traffic_resampled = traffic.groupby(['IMEI']).apply(lambda x: x.resample('1h', on='ds').sum().reindex(t_index).fillna(0))
print(traffic_resampled)

                                          IMEI         y
IMEI                                                    
0000****0000 2018-09-01 00:00:00  0000****0000  5.575131
             2018-09-01 01:00:00             0  0.000000
             2018-09-01 02:00:00             0  0.000000
             2018-09-01 03:00:00             0  0.000000
             2018-09-01 04:00:00             0  0.000000
...                                        ...       ...
a100****f758 2018-09-30 19:00:00             0  0.000000
             2018-09-30 20:00:00             0  0.000000
             2018-09-30 21:00:00             0  0.000000
             2018-09-30 22:00:00             0  0.000000
             2018-09-30 23:00:00             0  0.000000

[49428000 rows x 2 columns]


In [ ]:
traffic_resampled.reset_index(inplace=True)

In [ ]:
traffic_resampled = traffic_resampled.rename(columns={"level_1": "ds"})

In [ ]:
traffic_resampled_coor = traffic_resampled.set_index(['IMEI', 'ds'])
# 设置两个索引值，将原始表格数据形成多层索引
traffic_resampled_coor = traffic_resampled_coor.unstack(level=-1)
# unstack()即stack()的反操作，将行旋转到列，即把竖向排列的数据横过来，参数是层级序号

In [ ]:
exe = traffic_resampled_coor.iloc[:,0:24]
# iloc[a,b]表示取第a行第b列的数据

In [ ]:
sns.set(rc={'figure.figsize':(15,8.75)})
#sns.heatmap(X.corr(), annot=True);
fig = plt.figure()
ax = fig.add_subplot(111)
# 创建子图，参数三个数表示共有1*1个子图，这是第1个
cax = ax.matshow(exe.corr(), interpolation='nearest')
# matshow()表示绘制矩阵，根据颜色表示矩阵中数值大小
# corr表示计算相关系数矩阵
fig.colorbar(cax)
plt.show();